In [ ]:
startup

In [ ]:
% main.m

% This script loads the provided data, loads (or creates SENSE maps) and
% reconstructs the data for different undersampling factors (R values).
% Reconstructions are done in the CreateFigure functions which are meant to
% replicate the Figures of the original paper.

clear;
paths = setupPaths();
doLoadSenseMap = 1;

%% Load Data
% Possible Datasets are: 'brain', 'heart'
properties.dataset = 'brain';
data = loadData(paths.root, properties.dataset);

%% Set up properties
properties.Nimg = data.Nimg;        % Number of voxels N
properties.doSense = 1;             % Do SENSE reconstruction?
properties.R = 2;                   % Undersampling factor (positive integer or range)
properties.nIterations = 10;        % Number of CG-SENSE iterations
properties.gridding.os = 2;         % Gridding oversampling factor
properties.gridding.width = 4;      % Gridding kernel width as a multiple of dk without oversampling
properties.getSCdata = 0;           % Return single-coil images?
properties.doVis = 0;               % Plot current image after each CG iteration?
properties.saveIterSteps = 0;       % Store images from all CG iterations?
properties.doNoiseCov = 1;          % Use noise covariance matrix?
properties.calculateDelta = 0;      % Calculate delta (normed difference of each iteration step to reference)?
properties.dokspaceApodization = 0; % Add additional k-space apodization? (used for SENSE map creation)
% computation method for kspace filter
% 'gridding' - gridding of 1s on trajectory (for arbitrary traj) & mask
% 'disk'     - use circular disk (only useful for radials/spirals)
properties.kSpaceFilterMethod = 'gridding'; 

%% Calculate or load SENSE map
data.sense = getSenseMap(paths, properties, doLoadSenseMap, data);

In [ ]:
%plot gnuplot

% createFigure4.m

% This function creates Figure 4 of the original paper (log(delta) over
% number of iterations). It first sets properties that are held constant for
% all of the following reconstructions. They are very similar to the main
% script. In the first step a reconstruction using all data (R=1) is made
% with 5 iterations. The number of iterations was chosen manually as we
% found that the brain dataset showed a good reconstruction after 5 steps.
% Then, reconstructions with undersampling factors of R=2,3,4,5 are done. A
% porperty called 'calculateDelta' is used and set to 1 to output the delta
% for each iteration step.

properties.Nimg = data.Nimg;        
properties.gridding.os = 2;  
properties.gridding.width = 4;   
properties.doVis = 0;            
properties.saveIterSteps = 0;  
properties.doNoiseCov = 1;
properties.getSCdata = 0;
properties.calculateDelta = 0;
properties.dokspaceApodization = 0;

% R=1 image
properties.doSense = 1;
properties.R = 1;
properties.nIterations = 5;
properties.kSpaceFilterMethod = 'gridding'; 

out = iterativeRecon(data, properties);
reference.image = out.imageComb;
mask_tmp = zeros(size(reference.image));
mask_tmp(abs(reference.image) > mean(mean(abs(reference.image)))) = 1;
se = strel('diamond',2);
mask_tmp = imopen(mask_tmp, se);
reference.mask = mask_tmp;

% Range of R
R = [1, 2, 3, 4, 5];
properties.nIterations = 30;
deltas = zeros(properties.nIterations+1, length(R));
Deltas = zeros(properties.nIterations+1, length(R));
for ic1=1:length(R)
    disp(['Reconstruct image with R = ' num2str(R(ic1)) '. (' num2str(ic1) '/' num2str(length(R)) ')']);
    properties.R = R(ic1);
    properties.saveIterSteps = 0;
    properties.doNoiseCov = 1;
    properties.getSCdata = 0;
    properties.calculateDelta = 1;
    
    out_tmp = iterativeRecon(data, properties, reference);
    deltas(:,ic1) = out_tmp.deltas;
    Deltas(:,ic1) = out_tmp.Deltas;
end

nIterations = size(Deltas,1)-1;


In [ ]:
% Save each curve as a variable
iterations = 0:nIterations;
Deltas2=log10(Deltas(:,2));
Deltas3=log10(Deltas(:,3));
Deltas4=log10(Deltas(:,4));
Deltas5=log10(Deltas(:,5));

deltas1 = log10(deltas(:,1));
deltas2 = log10(deltas(:,2));
deltas3 = log10(deltas(:,3));
deltas4 = log10(deltas(:,4));
deltas5 = log10(deltas(:,5));

In [ ]:
%use sos
%get iterations --from Octave
%get Deltas2 --from Octave
%get Deltas3 --from Octave
%get Deltas4 --from Octave
%get Deltas5 --from Octave
%get deltas1 --from Octave
%get deltas2 --from Octave
%get deltas3 --from Octave
%get deltas4 --from Octave
%get deltas5 --from Octave

In [ ]:
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML


In [ ]:
sub1_2 = go.Scatter(
    x = iterations,
    y = Deltas2,
    name = 'R=2',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(217,83,25)'),
    showlegend=False,
)

sub1_3 = go.Scatter(
    x = iterations,
    y = Deltas3,
    name = 'R=3',
    text = 'R=3',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(237,177,32)'),
    showlegend=False,
)

sub1_4 = go.Scatter(
    x = iterations,
    y = Deltas4,
    name = 'R=4',
    text = 'R=4',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(126,47,142)'),
    showlegend=False,
)

sub1_5 = go.Scatter(
    x = iterations,
    y = Deltas5,
    name = 'R=5',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(119,172,48)'),
    showlegend=False,
)


In [ ]:
sub2_1 = go.Scatter(
    x = iterations,
    y = deltas1,
    name = 'R=1',
    text = 'R=1',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(0,114,189)'),
)

sub2_2 = go.Scatter(
    x = iterations,
    y = deltas2,
    name = 'R=2',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(217,83,25)'),
)

sub2_3 = go.Scatter(
    x = iterations,
    y = deltas3,
    name = 'R=3',
    text = 'R=3',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(237,177,32)'),
)

sub2_4 = go.Scatter(
    x = iterations,
    y = deltas4,
    name = 'R=4',
    text = 'R=4',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(126,47,142)'),
)

sub2_5 = go.Scatter(
    x = iterations,
    y = deltas5,
    name = 'R=5',
    text = 'R=2',
    hoverinfo = 'x+y+text',
    line=dict(color='rgb(119,172,48)'),
)


In [ ]:
from plotly import tools

fig = tools.make_subplots(rows=1, cols=2, print_grid=False)
fig.append_trace(sub1_2, 1, 1)
fig.append_trace(sub1_3, 1, 1)
fig.append_trace(sub1_4, 1, 1)
fig.append_trace(sub1_5, 1, 1)

fig.append_trace(sub2_1, 1, 2)
fig.append_trace(sub2_2, 1, 2)
fig.append_trace(sub2_3, 1, 2)
fig.append_trace(sub2_4, 1, 2)
fig.append_trace(sub2_5, 1, 2)


layout_subplot = go.Layout(
    width=800,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    annotations=[
        dict(
            x=0.15,
            y=-0.175,
            showarrow=False,
            text='Iterations',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.85,
            y=-0.175,
            showarrow=False,
            text='Iterations',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.50,
            showarrow=False,
            text='log<sub>10</sub>Δ<sub><i>approx</i></sub>',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.49,
            y=0.50,
            showarrow=False,
            text='log<sub>10</sub>δ',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2,
        domain=[0, 0.45]
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2,
        domain=[0, 1],
    ),
    xaxis2=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2,
        domain=[0.55, 1],
    ),
    yaxis2=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2,
        domain=[0, 1],
        anchor='x2'
    ),
    legend=dict(
        x=0.9,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig["layout"]=layout_subplot
iplot(fig)